## From EXR to YUV
* .exr images from ftp://svtopencontent.svt.se
* 1500 images at 50 fps = 30sec
* inside a .yuv raw video with 4k resolution and yuv420p sampling

In [ ]:
import subprocess
c_seq = "ffmpeg -y -start_number 180000 -framerate 50 -i test_vids/smithy/smithy_%04d.exr \
        -c:v rawvideo -pix_fmt yuv420p -s 3840x2160 -r 50 test_vids/smithy.yuv"
subprocess.call(c_seq, shell=True)

## From YUV to YUV multires
the .yuv reference uncompressed file is downsized to 5 more resolutions

In [ ]:
import numpy as np
import subprocess
source = "smithy"
res_widths = np.array([3840, 2560, 1920, 1280, 1024, 640])
res_heights = np.array([2160, 1440, 1080, 720, 576, 360])
for i in range(1, res_widths.size):
    path = "test_vids/downsampled/" + source + "_" + str(res_widths[i-1]) + ".yuv"
    out = "test_vids/downsampled/" + source + "_" + str(res_widths[i]) + ".yuv"
    command = f"ffmpeg -f rawvideo -video_size {res_widths[i-1]}x{res_heights[i-1]} -r 50 -pixel_format yuv420p \
    -i {path} -f rawvideo -c:v rawvideo -filter:v scale={res_widths[i]}x{res_heights[i]} -pix_fmt yuv420p {out}"
    subprocess.call(command, shell=True)

## From YUV multires to H264 AVC

In [ ]:
import os
import subprocess
folder = 'test_vids/downsampled'
source = "smithy"
res_widths = [3840, 2560, 1920, 1280, 1024, 640]
res_heights = [2160, 1440, 1080, 720, 576, 360]
quality = [1, 11, 23, 34, 50]
for file in os.listdir(folder):
    path = os.path.join(folder, file)
    if str(file).endswith('.yuv'):
        current_res = str(file)[:-4].strip(source + "_")
        current_index = res_widths.index(int(current_res))
        for current_crf in range(0,5):
            out = "test_vids/encoded/" + source + "AVC_" + str(current_res) + "_cf" + str(quality[current_crf]) + ".mp4"
            to_avc = f"ffmpeg -f rawvideo -video_size {res_widths[current_index]}x{res_heights[current_index]} \
            -r 50 -pixel_format yuv420p -i {path} -c:v libx264 -crf {quality[current_crf]} {out}"
            subprocess.call(to_avc, shell=True)

## From YUV multires to H265 HEVC